<a href="https://colab.research.google.com/github/everestso/47.53.59/blob/master/ElDoradoRumIowaGCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring El Dorado Rum Sales in Iowa w/ GCP


In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

#ensure the file is accessible
!ls /content/gdrive/'My Drive'/'csuf'/'projects'/'GCP'

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/gdrive/My Drive/csuf/projects/GCP/c264s20-557a598add94.json"

#ensure the path is set correctly
!echo $GOOGLE_APPLICATION_CREDENTIALS

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
c264s20-557a598add94.json  GCP.BigData.ML.Wk1Links.gdoc
GCP.BigData.ML.gdoc	   GCP.CoreInfrastructure.gdoc
GCP.BigData.ML.Lab2.gdoc   GCP.ML.Special.gdoc
GCP.BigData.ML.Lab3.gdoc   GCP.Notes.gdoc
GCP.BigData.ML.Lab4.gdoc   public_datasets_one_pager.pdf
/content/gdrive/My Drive/csuf/projects/GCP/c264s20-557a598add94.json


In [22]:
from google.cloud import bigquery
client = bigquery.Client()

In [23]:
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys')
dfprint = lambda df:print(pdtabulate(df))

In [24]:
sql = """
SELECT 
  item_description
  ,ROUND(SUM(volume_sold_gallons),2) AS gallons_sold
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE item_description like "%El Dorado%"
GROUP BY 1
ORDER BY 2 DESC
"""
df = client.query(sql).to_dataframe()

In [25]:
dfprint(df)

    item_description                             gallons_sold
--  -----------------------------------------  --------------
 0  El Dorado Special Reserve 12YR                      27.29
 1  El Dorado 15 Year Old Special Reserve Rum           22.5
 2  El Dorado 8YR                                       20.15
 3  El Dorado Special Reserve Rum                       13.09
 4  El Dorado 21YR Special Reserve Rum                   3.55
 5  El Dorado 3 Year Old Rum                             2.37


In [26]:
sql = """
SELECT 
  store_number
  , city
  , extract(year from date) as year
  , sum( volume_sold_gallons )
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
where item_description like "%El Dorado%" 
group by 1, 2, 3
order by 4
"""
df = client.query(sql).to_dataframe()

In [27]:
dfprint(df)

      store_number  city            year    f0_
--  --------------  ------------  ------  -----
 0            5444  Marshalltown    2020   0.59
 1            3820  Sioux City      2019   1.18
 2            2248  Des Moines      2019   1.18
 3            5251  Mason City      2019   1.18
 4            4129  AMES            2016   1.19
 5            3896  Iowa City       2018   1.19
 6            3896  Iowa City       2016   1.19
 7            2285  IOWA CITY       2013   1.19
 8            4129  Ames            2020   2.36
 9            3773  Cedar Rapids    2017   2.38
10            4129  AMES            2012   2.38
11            5463  Decorah         2019   3.55
12            5463  Decorah         2018   3.55
13            4129  AMES            2013   3.57
14            4129  Ames            2016   3.57
15            4129  Ames            2017   4.76
16            4129  Ames            2018   4.76
17            4129  Ames            2019   7.11
18            2190  Des Moines      2018

In [28]:
sql = """
SELECT distinct
  store_number
  , store_name
  , address
  , city
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
#where extract(year from date) = 2019 and extract(month from date)=11
where store_number in unnest([
"5251", "3820",
"2285", "3896",
"3773", "5463",
"4129", "2190"
])
"""
df = client.query(sql).to_dataframe()

In [29]:
dfprint(df.sort_values(by=['store_number','address']))

      store_number  store_name                                   address               city
--  --------------  -------------------------------------------  --------------------  ------------
 0            2190  Central City Liquor, Inc.                    1460 2ND AVE          Des Moines
 2            2190  Central City Liquor, Inc.                    1460 2ND AVE          DES MOINES
16            2190  Central City Liquor, Inc.                    1460 2nd Ave          Des Moines
 3            2285  John's Grocery                               401 EAST MARKET ST    IOWA CITY
 9            2285  John's Grocery                               401 East Market St    Iowa City
 1            3773  Benz Distributing                            501 7TH AVE SE        CEDAR RAPIDS
 5            3773  Benz Distributing                            501 7th Ave SE        Cedar Rapids
10            3820  Charlie's Wine and Spirits,                  507 W 19th St         SIOUX CITY
14            3820  Ch